In [4]:
#!/usr/bin/env python
# coding: utf-8

"""
Program: Ground Deformation Rate Analysis
Description: This script processes compaction data from an HDF5 file, 
computes deformation rates at different depths, and visualizes them using 
a horizontal bar chart where positive values are displayed on the left 
and negative values on the right.
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from appgeopy import *
from my_packages import *

# -----------------------------------------------
# Load HDF5 file containing compaction data
# -----------------------------------------------
dataset_path = r"D:\1000_SCRIPTS\003_Project002\20241016_MLCW_to_HDF5\20241105_MLCW_CRFP_v6.h5"
data, metadata = gwatertools.open_HDF5(dataset_path)

# Select a station for analysis
stations = list(data.keys())
# station_name = "ANHE"

for station_name in tqdm(stations):
    try:
        # -----------------------------------------------
        # Extract and process time series data
        # -----------------------------------------------
        time_arr = list(map(lambda x: x.decode(), data[station_name]["date"]))  # Decode byte strings
        time_arr = pd.to_datetime(time_arr)  # Convert to pandas datetime format
        
        # Extract depth values corresponding to measurements
        depth_arr = data[station_name]["depth"]
        
        # Retrieve compaction data (measured relative to base reference)
        compact_ref2base = data[station_name]["values"]["ref2base"]
        
        # Convert data into a pandas DataFrame and preprocess it
        compact_df = pd.DataFrame(data=compact_ref2base, columns=time_arr)  # Create DataFrame
        compact_df = compact_df.loc[:, "2014":"2022"]  # Select data within the specified time range
        compact_df = compact_df * 1000  # Convert meters to millimeters
        compact_byDepth = compact_df.diff(-1)  # Compute year-over-year compaction differences
        compact_byDepth.iloc[-1, :] = compact_df.iloc[-1, :]
        compact_byDepth = compact_byDepth.sub(compact_byDepth.iloc[:, 0], axis=0)
        
        # Convert time data to numerical format for trend estimation
        time_array_numeric = (compact_byDepth.columns - compact_byDepth.columns[0]).days.to_numpy()
        fulltime_array = get_fulltime(compact_byDepth.columns)
        fulltime_array_numeric = (fulltime_array - fulltime_array[0]).days.to_numpy()
        
        # -----------------------------------------------
        # Compute deformation rate (velocity) at each depth
        # -----------------------------------------------
        velocity_arr = []
        for idx, row in compact_byDepth.iterrows():
            # Compute linear velocity trend
            linear_trend, coeffs = analysis.get_polynomial_trend(
                x=time_array_numeric, y=row.to_numpy(), order=1, x_estimate=fulltime_array_numeric
            )
            linear_velocity = coeffs[-1] * 365.25  # Convert to mm/year
            velocity_arr.append(linear_velocity)
        
        # -----------------------------------------------
        # Data preparation for visualization
        # -----------------------------------------------
        # Convert lists to numpy arrays
        x = np.array(velocity_arr)
        y = np.array(depth_arr)
        
        # Sort data for better visualization
        sorted_indices = np.argsort(y)
        x_sorted = x[sorted_indices]
        y_sorted = y[sorted_indices]
        
        # Define colors for positive (blue) and negative (red) values
        colors = np.where(x_sorted >= 0, "blue", "red")
        
        # -----------------------------------------------
        # Create and configure the figure
        # -----------------------------------------------
        fig = plt.figure(figsize=(8.3, 11.7 * 2 / 3))
        ax = fig.add_subplot(111)  # Create a single subplot
        
        plt.style.use("seaborn-v0_8-bright")
        
        # Plot horizontal bar chart
        ax.barh(y_sorted, x_sorted, color=colors, edgecolor="none", alpha=1, height=3)
        
        # Configure x-axis: Positive values on the left, Negative values on the right
        ax.axvline(0, color="black", linewidth=1)  # Add a central vertical line
        ax.set_xlabel("Deformation Rate (mm/year)", fontsize=14, fontweight="bold")
        ax.set_ylabel("Depth (m)", fontsize=14, fontweight="bold")
        
        # Improve tick labels for better readability
        ax.tick_params(axis="x", labelsize=12)
        ax.tick_params(axis="y", labelsize=12)
        
        # Remove unnecessary spines
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        
        # Set y-axis limits dynamically based on depth values
        if max(depth_arr) < 300:
            ax.set_ylim(-5, 310)
        else:
            ax.set_ylim(-5, max(depth_arr) + 5)
        
        # -----------------------------------------------
        # Invert axes for correct visualization
        # -----------------------------------------------
        ax.invert_yaxis()  # Ensure depth values increase downwards
        ax.invert_xaxis()  # Reverse x-axis so negative values appear on the right
        
        # -----------------------------------------------
        # Apply additional axis formatting
        # -----------------------------------------------
        visualize.configure_axis(
            ax,
            xlabel="Deformation Rate (mm/year)",
            ylabel="Depth (m)",
            title=station_name,
            tick_direction="out",
            major_tick_length=10,
            minor_tick_length=5,
        )
        visualize.configure_ticks(ax, y_major_interval=50, y_minor_interval=10)
        ax.grid(which="major", axis="y", color="lightgrey", linewidth=0.5, alpha=0.7)
        
        savename = f"MLCW_barplot/{station_name}_bar.png"
        visualize.save_figure(fig, savename)
        # Show the final plot
        plt.close()
    except Exception as e:
        print(station_name, e)
        pass

  0%|          | 0/59 [00:00<?, ?it/s]

DAZHUANG1 'date'
